In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        # print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)
        
def query(q):
    conn = create_connection('sqlite_db_pythonsqlite.db')
    cur = conn.cursor()
    cur.execute(q)
    rows = cur.fetchall() 
    return rows


def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()
    


2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


In [2]:
Q10 = """
SELECT facility, round(sum(cost),2) as revenue
FROM(
    SELECT f.name as facility, 
      CASE WHEN m.firstname = 'GUEST' THEN f.guestcost * b.slots
      ELSE f.membercost * b.slots END AS cost
      FROM Members AS m
      JOIN Bookings AS b ON m.memid = b.memid
      JOIN Facilities AS f ON b.facid = f.facid
      ) subquery
GROUP BY facility
ORDER BY revenue DESC
;
"""
print(pd.DataFrame(query(Q10),columns=['facility', 'revenue']))

          facility  revenue
0   Massage Room 1  50351.6
1   Massage Room 2  14454.6
2   Tennis Court 2  14310.0
3   Tennis Court 1  13860.0
4     Squash Court  13468.0
5  Badminton Court   1906.5
6       Pool Table    270.0
7    Snooker Table    240.0
8     Table Tennis    180.0


In [3]:
Q11 = """
SELECT m1.surname || ", " || m1.firstname, m2.surname || ", " || m2.firstname
FROM MEMBERS AS m1
JOIN MEMBERS AS m2 ON m1.recommendedby = m2.memid
WHERE m1.recommendedby != ''
ORDER BY m1.surname ASC, m1.firstname ASC;
"""
print(pd.DataFrame(query(Q11),columns=['Member','Recruited By']))

                      Member        Recruited By
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8           Genting, Matthew     Butters, Gerald
9                 Hunt, John  Purview, Millicent
10              Jones, David    Joplette, Janice
11            Jones, Douglas        Jones, David
12          Joplette, Janice       Smith, Darren
13           Mackenzie, Anna       Smith, Darren
14             Owen, Charles       Smith, Darren
15             Pinker, David     Farrell, Jemima
16        Purview, Millicent        Smith, Tracy
17         Rumney, Henrietta    Genting, Matthew
18         Sarwin, Ramnaresh     Bader, Florence
19               Smi

In [4]:
Q12 = """
SELECT facility, name, sum(usage) FROM 
    (SELECT f.name AS facility, m.surname || ", " || m.firstname AS name, b.slots / 2.0 AS usage
     FROM FACILITIES AS f
     JOIN BOOKINGS AS b ON f.facid = b.facid
     JOIN MEMBERS AS m ON b.memid = m.memid
     WHERE m.memid != 0) 
GROUP BY name, facility
ORDER BY facility
"""

print(pd.DataFrame(query(Q12), columns = ['Facility','Member','Usage (hours)']).pivot(index='Member',columns='Facility').fillna(0))

                           Usage (hours)                                \
Facility                 Badminton Court Massage Room 1 Massage Room 2   
Member                                                                   
Bader, Florence                     13.5            0.0            2.0   
Baker, Anne                         15.0            3.0            2.0   
Baker, Timothy                      10.5           25.0            0.0   
Boothe, Tim                         18.0           38.0            0.0   
Butters, Gerald                     31.5           33.0            1.0   
Coplin, Joan                         0.0            1.0            2.0   
Crumpet, Erica                       3.0            2.0            0.0   
Dare, Nancy                         15.0           19.0            5.0   
Farrell, David                       0.0            0.0            0.0   
Farrell, Jemima                     10.5           34.0            0.0   
Genting, Matthew                     0

In [5]:
Q13 = """
SELECT facility, month, sum(usage) FROM 
    (SELECT f.name AS facility, strftime('%m', DATE(b.starttime)) AS month, b.slots / 2.0 AS usage
     FROM FACILITIES AS f
     JOIN BOOKINGS AS b ON f.facid = b.facid
     JOIN MEMBERS AS m ON b.memid = m.memid
     WHERE m.memid != 0) 
GROUP BY month, facility
ORDER BY month, facility
"""

print(pd.DataFrame(query(Q13), columns = ['Facility','Month', 'Usage (hours)']).pivot(index='Facility',columns='Month'))

                Usage (hours)              
Month                      07     08     09
Facility                                   
Badminton Court          82.5  207.0  253.5
Massage Room 1           83.0  158.0  201.0
Massage Room 2            4.0    9.0   14.0
Pool Table               55.0  151.5  221.5
Snooker Table            70.0  158.0  202.0
Squash Court             25.0   92.0   92.0
Table Tennis             49.0  148.0  200.0
Tennis Court 1          100.5  169.5  208.5
Tennis Court 2           61.5  172.5  207.0
